In [2]:
import time, sys, math, copy
import numpy as np
from matplotlib import pyplot as plt 

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import import_ipynb
from MoveData import Options, json2datatools, num_batches, nopeak_mask, create_masks
%matplotlib inline

importing Jupyter notebook from MoveData.ipynb


If you are not using this notebook to learn, change the below variable `teaching` to `False` so that other notebooks can import the functions defined in this notebook without running all the examples, if you are here to learn and interact with the notebook, change it to `True`

In [3]:
teaching = False # True 

Before going into this lesson, lets remind ourselves where the components we are about to learn fit into the big picture. Recall the overall architecture of the Transformer. It is useful to image yourself as a piece of data, then imagine the journey you are about to go on. 

Imagine you emerge from the `data_iter` function as a sequence of integers shaped (batch size, sequence length)

The first place you will go is the encoder. It is initialized using `self.encoder = Encoder(in_vocab_size, emb_dim, n_layers, heads, dropout)` and used in the form `e_output = self.encoder(src_seq, src_mask)`. 

In [4]:
class Transformer(nn.Module):
    def __init__(self, in_vocab_size, out_vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.encoder = Encoder(in_vocab_size, emb_dim, n_layers, heads, dropout)
        self.decoder = Decoder(out_vocab_size, emb_dim, n_layers, heads, dropout)
        self.out = nn.Linear(emb_dim, out_vocab_size)
    def forward(self, src_seq, src_mask, trg_seq,  trg_mask):
        e_output = self.encoder(src_seq, src_mask)
        d_output = self.decoder(trg_seq, trg_mask, e_output, src_mask)
        output = self.out(d_output)
        return output

The first component, or module, within the Encoder, is the Embedder. emb_dim is short for embedding_dimensions

`self.embed = Embedder(vocab_size, embedding_dimensions)` 

`x = self.embed(source_sequence)`

In [5]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.n_layers = n_layers
        self.embed = Embedder(vocab_size, emb_dim)
        self.pe = PositionalEncoder(emb_dim, dropout=dropout)
        self.layers = get_clones(EncoderLayer(emb_dim, heads, dropout), n_layers)
        self.norm = Norm(emb_dim)
    def forward(self, src_seq, mask):
        x = self.embed(src_seq)
        x = self.pe(x)
        for i in range(self.n_layers):
            x = self.layers[i](x, mask)
        x = self.norm(x)
        return x

## Embedding

What is Embedding? As we mentioned before, one ability, or limitation depending on how you look at it, of chloe is her fixed vocabulary, each word or symbol in her vocabulary is assigned an integer. For example the word hi is assigned 3, the word dog is 17, a word not in the vocabulary is 0. This integer is the `token` index. 

The neural nework sees every word as a vector. [A vector of 3 real numbers forms the coordinates in 3D space](https://youtu.be/fNk_zzaMoSs). We use several more dimensions than 3 in deep learning, if we use 512 dimensions, our `embedding_dimensions = 512`, this means that each word is a point in 512 dimensional space. The same concepts apply to 3D space in that the location of that word in 3D space tells you it's [meaning and meaning relative to other words](https://youtu.be/8rXD5-xhemo?t=1550).

<img src="../saved/images/wordvectors.png" height=400 width=400>

In the image you see above, similar words are close to each other, not only that, the direction they are separated from eachother also carries meaning. In the image, there are 3 clusters of words and the separation between them has something to do with age/time/etc.  

If you stack all the vectors on top of eachother row by row, you get a matrix. Remember how each word is represented by both a vector and an integer? well this integer is the index for a row in the matrix. The matrix is called the embedding matrix. you might say that we "embed" words into the matrix. Now for the example, run the cells below 

In [7]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.emb_dim = emb_dim
        self.embed = nn.Embedding(vocab_size, emb_dim)
    def forward(self, x):
        return self.embed(x)

To demonstrate the Embedder, I will show you it's two functions

- storing an embedding matrix of word vectors 
- transforming a sequence of integers that represent token indices, into a sequence of vectors

Lets start off by creating a toy embedding with only 2 tokens in it. These tokens will be represented in 4-dimensional space. The data comes to us as a sequence of integers. 

After creating the embedding, I pass into the Embedder a sequence of token indices, the integers `[1,0,1]` created by this line `source_sequence = torch.from_numpy(np.asarray([1,0,1])).unsqueeze(0)`, the ending `.unsqueeze(0)` adds a batch dimension and thus turns the shape of the input from `[3,]` to `[1,3]`. The information in the tensor is the same, The embedder just requires it's input to be of shape (batch_size, sequence_length)

In [9]:
if teaching:  
    torch.manual_seed(0)
    embedding = Embedder(vocab_size=2, emb_dim=4)
    source_sequence = torch.from_numpy(np.asarray([1,0,1])).unsqueeze(0)
    print('source_sequence',source_sequence, source_sequence.shape)
    print("---------------------------------------------------")
    print("Embedding Matrix", embedding.embed.weight, embedding.embed.weight.shape)
    print("---------------------------------------------------")
    sequence_of_vectors = embedding(source_sequence)
    print('sequence_of_vectors')
    print(sequence_of_vectors, sequence_of_vectors.shape)

source_sequence tensor([[1, 0, 1]]) torch.Size([1, 3])
---------------------------------------------------
Embedding Matrix Parameter containing:
tensor([[ 1.5410, -0.2934, -2.1788,  0.5684],
        [-1.0845, -1.3986,  0.4033,  0.8380]], requires_grad=True) torch.Size([2, 4])
---------------------------------------------------
sequence_of_vectors
tensor([[[-1.0845, -1.3986,  0.4033,  0.8380],
         [ 1.5410, -0.2934, -2.1788,  0.5684],
         [-1.0845, -1.3986,  0.4033,  0.8380]]], grad_fn=<EmbeddingBackward>) torch.Size([1, 3, 4])


## Embedding Matrix

Suppose, as I mentioned earlier, I am unhappy with chloe's limited vocabulary `{"me":0, "give":1}`. I want her to learn the word "covfefe". I will have to add a word to her dictionary `{"covfefe":2}` (not shown), then I will have to initialize a new word vector and add it, concatenate it, to chloe's embedding matrix `embedding.embed.weight`. In the cell below, I do just that. 

`concatenated_matrix` is the concatenation of the old matrix `embedding.embed.weight` with the `new_vector` appended to the bottom row. Now we can embed our

sequence of integers **(batch_size, sequence_length)** 

into a vector sequence for the phrase "give me covfefe"


In [10]:
if teaching:  
    np.random.seed(0)
    new_vector = torch.from_numpy(np.random.uniform(-0.1,0.1,(1, 4)).astype(np.float32))
    print('new_vector', new_vector, new_vector.shape)
    print("---------------------------------------------------")
    concatenated_matrix = torch.cat((embedding.embed.weight, new_vector),dim=0)
    embedding.embed.weight=nn.Parameter(concatenated_matrix,requires_grad=True)
    print("New Embedding Matrix", embedding.embed.weight, embedding.embed.weight.shape)
    print("---------------------------------------------------")
    source_sequence = torch.from_numpy(np.asarray([1,0,2])).unsqueeze(0)
    print('source_sequence',source_sequence, source_sequence.shape)
    print("---------------------------------------------------")
    sequence_of_vectors = embedding(source_sequence)
    print('sequence_of_vectors')
    print(sequence_of_vectors, sequence_of_vectors.shape)

new_vector tensor([[0.0098, 0.0430, 0.0206, 0.0090]]) torch.Size([1, 4])
---------------------------------------------------
New Embedding Matrix Parameter containing:
tensor([[ 1.5410, -0.2934, -2.1788,  0.5684],
        [-1.0845, -1.3986,  0.4033,  0.8380],
        [ 0.0098,  0.0430,  0.0206,  0.0090]], requires_grad=True) torch.Size([3, 4])
---------------------------------------------------
source_sequence tensor([[1, 0, 2]]) torch.Size([1, 3])
---------------------------------------------------
sequence_of_vectors
tensor([[[-1.0845, -1.3986,  0.4033,  0.8380],
         [ 1.5410, -0.2934, -2.1788,  0.5684],
         [ 0.0098,  0.0430,  0.0206,  0.0090]]], grad_fn=<EmbeddingBackward>) torch.Size([1, 3, 4])


## repeating modules with nn.ModuleList

`get_clones()` allows us to stack the same module N number of times. The data would thus pass through multiple modules of the same repeating architecture. To be clear, the weights would be NOT be shared though, each module is allowed to compute something different. 

FeedForward is a vanilla feed forward neural network that has the same dimensions for it's input as for it's output. If you need a brush up on vanilla feed forward neural networks, I suggest []() or [Welch Labs](https://youtu.be/bxe2T-V8XRs) or [3Blue1Brown](https://youtu.be/aircAruvnKk). One small modification I made here is using the leaky_relu rather than relu activation function. 

In [15]:
def get_clones(module, N):
    '''
    example usage:
        
        # initialize n_layers deep copies of the same encoder
        self.layers = get_clones(EncoderLayer(emb_dim, heads, dropout), n_layers)
        
        # usage, apply n_layers transformations to x
        for i in range(self.n_layers):
            x = self.layers[i](x, mask)  
    '''
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class FeedForward(nn.Module):
    def __init__(self, emb_dim, ff_dim=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(emb_dim, ff_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(ff_dim, emb_dim)
    
    def forward(self, x, explain=False):
        x = self.dropout(F.leaky_relu(self.linear_1(x)))
        if explain: print('hidden layer output',x)
        x = self.linear_2(x)
        return x

`self.linear_1 = nn.Linear(emb_dim, ff_dim)` creates one fully connected layer. Dropout is not a layer, it is simply a way to add robustness to the neural network by randomly zeroing out the outputs of one layer and proportionally scaling up the remainder of the outputs. This forces the next layer to not spread out it's dependence on the previous layers outputs rather than rely too heavily on a few of the previous layers dimensions.

The cell below recreates the feed forward neural network (FFNN) depicted in this diagram having an input of dimension 4, a hidden layer with output of dimension 5, and output of dimension 4.

<img src="../saved/images/feedforward.png">

With `explain=True` as an argument to the forward method of the FeedForward modules that is initiated with a dropout of 0.2, you would expect on average 1 of the 5 hidden layer outputs to be zeroed out to `0.0000` each time you run the forward pass. Run the next cell a couple times to observe this. 

In [16]:
if teaching:  
    FFNN = FeedForward(emb_dim = 4, ff_dim=5, dropout = 0.2)
    x = torch.from_numpy(np.asarray([.01,-.01,.01,-.01])).float().unsqueeze(0)
    print('output',FFNN(x, explain=True))

hidden layer output tensor([[-0.0037,  0.5284,  0.2887,  0.0000,  0.0000]], grad_fn=<MulBackward0>)
output tensor([[ 0.1001, -0.1847,  0.2447, -0.0275]], grad_fn=<AddmmBackward>)


Since the input and output dimensions of FeedForward are the same, I can stack any number of them repeatedly together to create an arbitrarily deep neural network. The cell below demonstrates this by demonstrating the use of the `get_clones()` funtion.

In [17]:
class repeatFeedForward(nn.Module):
    def __init__(self, repeats):
        super().__init__()
        self.repeats = repeats
        self.layers = get_clones(FeedForward(emb_dim=4, ff_dim=5, dropout=0.2), repeats)
    def forward(self, x, explain=True):
        for i in range(self.repeats):
            x = self.layers[i](x, explain=explain)
        return x

In [18]:
if teaching:  
    rFFNN = repeatFeedForward(repeats=3)
    x = torch.from_numpy(np.asarray([.01,-.01,.01,-.01])).float().unsqueeze(0)
    print('output',rFFNN(x, explain=True))

hidden layer output tensor([[-0.0000, -0.0029,  0.2470, -0.0000,  0.0000]], grad_fn=<MulBackward0>)
hidden layer output tensor([[ 0.0000, -0.0000,  0.0922, -0.0027,  0.6105]], grad_fn=<MulBackward0>)
hidden layer output tensor([[ 0.0000, -0.0000,  0.2158, -0.0047,  0.3337]], grad_fn=<MulBackward0>)
output tensor([[ 0.2067, -0.2059,  0.3808,  0.2397]], grad_fn=<AddmmBackward>)


## Normalization (Standardizing)

This concept is easier to teach in the domain of computer vision. The same concept applies to computer language. In both cases, the data are represented in vector space and both can benefit from normalization. 

Imagine you train a neural network to recognise pictures of your cat in your house. Then you try to test if it works and you realize that because all your training data was taken during the day, it doesnt recognize your cat as well during the evening, probably because the lighting is different. 

Thats because if you take one image that the model has learnt well and shift/scale the input features, the parameters are no longer optimal for this shift/scaled image. We normalize the input image to be between 0 and 1 or between -1 and 1 or a unit gaussian so that our first layer can learn faster by focusing on learning the variation within images rather than the variation between images.

This same intuition applies not just to the input. Since each layers output is another representation of the data and serves as the input to the next layer, this intuition applies to all the intermediate representations as well. 

In [16]:
class Norm(nn.Module):
    def __init__(self, emb_dim, eps = 1e-6):
        super().__init__()
        self.size = emb_dim
        # alpha and bias are learnable parameters that scale and shift
        # the representations respectively, aka stretch and translate 
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps #prevents divide by zero explosions 
    
    def forward(self, x):
        '''
        input: x, shape (batch size, sequence length, embedding dimensions)
        output: norm, shape (batch size, sequence length, embedding dimensions)
        '''
        norm = (x - x.mean(dim=-1, keepdim=True)) / (x.std(dim=-1, keepdim=True) + self.eps)
        norm = self.alpha * norm + self.bias
        return norm

In the image below, we take whatever the mean is mu and standard devation is sigma, and make it into a unit gaussian aka “standard normal” distribution.

<img src="https://miro.medium.com/max/744/1*GAcEj37smCOCZMrqp-rjjA.png">

The demo below uses a toy input of shape `[2, 2, 3]`, simulating a batch with 2 samples, 2 words in sequence length, and a embedding dimension of 3. 

The tensor above `x.mean(dim=-1, keepdim=True)` calculates the mean of each word vector and the tensor above `x.std(dim=-1, keepdim=True)` calculates the standard deviation of each word vector

If we make the simplifying assumption that our vector `x` follows a Gaussian distribution. We can subtract the mean and divide by the standard deviation, thereby obtaining a “standard normal” distribution, with mean 0 and standard deviation 1.


In [17]:
if teaching:
    x = np.asarray([ [[0.0, 1.0, 2.0], [2.0, 3.0, 4.0]], [[1.0, 2.0, 3.0], [3.0, 4.0, 5.0]]  ])
    x = torch.from_numpy(x).float()
    print(x, x.shape)
    print("------------------------------------")
    print(x.mean(dim=-1, keepdim=True))
    print("------------------------------------")
    print(x.std(dim=-1, keepdim=True))

tensor([[[0., 1., 2.],
         [2., 3., 4.]],

        [[1., 2., 3.],
         [3., 4., 5.]]]) torch.Size([2, 2, 3])
------------------------------------
tensor([[[1.],
         [3.]],

        [[2.],
         [4.]]])
------------------------------------
tensor([[[1.],
         [1.]],

        [[1.],
         [1.]]])
